# 03 - Prophet Modeling
Train a baseline Prophet model using the processed sales data and persist the fitted artifact for later phases.


## Outline
1. Import helpers and load configuration.
2. Bring in the daily aggregate prepared in Phase 1.
3. Find Best Hyperparameter
4. Fit Prophet with the shared pipeline helpers.
5. Save the model object for reuse.


In [1]:
import sys
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))


In [2]:
from src.utils import load_dataframe, load_yaml_config
from src.prophet_pipeline import run_pipeline, save_model
from src.prophet_config import search_best_prophet_config, save_prophet_config


In [3]:
data_path = Path("../data/processed/sales_daily.csv")
config_path = Path("../config/model_config.yaml")

daily_df = load_dataframe(data_path)
model_ready_df = daily_df.rename(columns={"InvoiceDate": "ds", "Sales": "y"})

### Prophet hyperparameter tuning

Validate a compact parameter grid on the latest six weeks before training the final model.


In [4]:

tuning_result = search_best_prophet_config(
    daily_df,
    holdout=42,
)
best_prophet_config = tuning_result["best_params"]
save_prophet_config(best_prophet_config)

pd.DataFrame(tuning_result["history"]).sort_values("mae").reset_index(drop=True)
print(f"Selected config (MAE={tuning_result['best_mae']:.2f}): {best_prophet_config}")


18:44:06 - cmdstanpy - INFO - Chain [1] start processing
18:44:06 - cmdstanpy - INFO - Chain [1] done processing
18:44:06 - cmdstanpy - INFO - Chain [1] start processing
18:44:06 - cmdstanpy - INFO - Chain [1] done processing
18:44:06 - cmdstanpy - INFO - Chain [1] start processing
18:44:06 - cmdstanpy - INFO - Chain [1] done processing
18:44:06 - cmdstanpy - INFO - Chain [1] start processing
18:44:06 - cmdstanpy - INFO - Chain [1] done processing
18:44:07 - cmdstanpy - INFO - Chain [1] start processing
18:44:07 - cmdstanpy - INFO - Chain [1] done processing
18:44:07 - cmdstanpy - INFO - Chain [1] start processing
18:44:07 - cmdstanpy - INFO - Chain [1] done processing
18:44:07 - cmdstanpy - INFO - Chain [1] start processing
18:44:07 - cmdstanpy - INFO - Chain [1] done processing
18:44:07 - cmdstanpy - INFO - Chain [1] start processing
18:44:07 - cmdstanpy - INFO - Chain [1] done processing
18:44:07 - cmdstanpy - INFO - Chain [1] start processing
18:44:08 - cmdstanpy - INFO - Chain [1]

Selected config (MAE=17624.92): {'seasonality_mode': 'multiplicative', 'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 5.0, 'holidays_prior_scale': 5.0, 'weekly_seasonality': True, 'daily_seasonality': False}


In [6]:
model, forecast = run_pipeline(model_ready_df, model_config=best_prophet_config, periods=90, freq="D")
model_path = save_model(model)

print(f"Model saved to: {model_path}")
forecast.tail()


18:44:15 - cmdstanpy - INFO - Chain [1] start processing
18:44:15 - cmdstanpy - INFO - Chain [1] done processing


Model saved to: ..\result\model.pkl


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
824,2012-03-04,39068.531714,2730.993087,40358.275738,37456.744855,40848.770030,-0.479523,-0.479523,-0.479523,-0.398885,-0.398885,-0.398885,-0.080639,-0.080639,-0.080639,0.0,0.0,0.0,20334.256024
825,2012-03-05,39118.864545,25932.875799,64181.186326,37477.797595,40935.552748,0.150255,0.150255,0.150255,0.249320,0.249320,0.249320,-0.099065,-0.099065,-0.099065,0.0,0.0,0.0,44996.654247
826,2012-03-06,39169.197376,31661.078157,68806.093645,37498.057177,41023.810482,0.291611,0.291611,0.291611,0.408861,0.408861,0.408861,-0.117250,-0.117250,-0.117250,0.0,0.0,0.0,50591.381153
827,2012-03-07,39219.530207,24535.912288,60717.953248,37510.759963,41106.061676,0.073181,0.073181,0.073181,0.207532,0.207532,0.207532,-0.134351,-0.134351,-0.134351,0.0,0.0,0.0,42089.662997
828,2012-03-08,39269.863038,32260.106755,69209.483869,37529.389080,41177.607009,0.294043,0.294043,0.294043,0.443622,0.443622,0.443622,-0.149579,-0.149579,-0.149579,0.0,0.0,0.0,50816.894754
